## Performance Metrics

Key performance indicators (KPIs):

- **Completion Rate**: The proportion of users who reach the final ‘confirm’ step.

  - According to this dataset, the completion rate is 67.5%.
    

- **Time Spent on Each Step**: The average duration users spend on each step.

- **Error Rates**: If there’s a step where users go back to a previous step, it may indicate confusion or an error. You should consider moving from a later step to an earlier one as an error.

### Code : performance metrics exploration

In [37]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import chi2_contingency
from scipy.stats.contingency import association
import functions as fun

%matplotlib inline

In [59]:
df_demo = pd.read_csv("../data/df_demo.csv")
df_exp = pd.read_csv("../data/df_experiment_tot.csv")
df_final = pd.read_csv("../data/df_final.csv")

In [60]:
merge_df = pd.merge(df_demo, df_exp, on = "client_id")

In [71]:
def completion_rate(df):
    
    confirms = df[ df["process_step"] == "confirm"]
    users_confirm = confirms["client_id"].nunique()
    tot_users = df["client_id"].nunique()

    completion_rate = round((users_confirm/tot_users)*100, 1)

    return completion_rate

In [70]:
completion_rate_exp = completion_rate(df_exp)
completion_rate_exp

67.5

**Average duration users spend on each step**

In [ ]:
# First function defined for time spent in each step
def time_steps_1(df):

    df["date_time"] = pd.to_datetime(df["date_time"])
    df["hour"] = df["date_time"].dt.hour
    df["minutes"] = df["date_time"].dt.minute
    df["seconds"] = df["date_time"].dt.second
    
    start_step1 = df[ df["process_step"].isin(["start","step_1"])]
    step1_step2 = df[ df["process_step"].isin(["step_1","step_2"])]
    step2_step3 = df[ df["process_step"].isin(["step_2","step_3"])]
    step3_confirm = df[ df["process_step"].isin(["step_3","confirm"])]
    
    
    #Time difference between Start and step1
    
    start_step1 = start_step1.sort_values(by = ["client_id", "date_time"])
    
    hours_spent_1 = (start_step1.groupby(["client_id","process_step"])["hour"].diff().mean())*3600 # converting the hours to seconds
    minutes_spent_1 = (start_step1.groupby(["client_id","process_step"])["minutes"].diff().mean())*60 # converting the minutes to seconds
    seconds_spent_1 = start_step1.groupby(["client_id","process_step"])["seconds"].diff().mean()

     #Time difference between Step1 and step2
    
    step1_step2 = step1_step2.sort_values(by = ["client_id", "date_time"])
    
    hours_spent_2 = (step1_step2.groupby(["client_id","process_step"])["hour"].diff().mean())*3600 # converting the hours to seconds
    minutes_spent_2 = (step1_step2.groupby(["client_id","process_step"])["minutes"].diff().mean())*60 # converting the minutes to seconds
    seconds_spent_2 = step1_step2.groupby(["client_id","process_step"])["seconds"].diff().mean()

    #Time difference between step2 and step3
    
    step2_step3 = step2_step3.sort_values(by = ["client_id", "date_time"])
                       
    hours_spent_3 = (step2_step3.groupby(["client_id","process_step"])["hour"].diff().mean())*3600 # converting the hours to seconds
    minutes_spent_3 = (step2_step3.groupby(["client_id","process_step"])["minutes"].diff().mean())*60 # converting the minutes to seconds
    seconds_spent_3 = step2_step3.groupby(["client_id","process_step"])["seconds"].diff().mean()
    
    #Time difference between step3 and confirm
    
    step3_confirm = step3_confirm.sort_values(by = ["client_id", "date_time"])
    
    hours_spent_4 = (step3_confirm.groupby(["client_id","process_step"])["hour"].diff().mean())*3600 # converting the hours to seconds
    minutes_spent_4 = (step3_confirm.groupby(["client_id","process_step"])["minutes"].diff().mean())*60 # converting the minutes to seconds
    seconds_spent_4 = step3_confirm.groupby(["client_id","process_step"])["seconds"].diff().mean()
    
    
    print(f"""The average activity duration of clients is:
    Between Start and Step_1: {hours_spent_1 + minutes_spent_1 + seconds_spent_1: .2f} seconds
    Between Step_1 and Step2: {hours_spent_2 + minutes_spent_2 + seconds_spent_2: .2f} seconds
    Between Step_2 and Step3: {hours_spent_3 + minutes_spent_3 + seconds_spent_3: .2f} seconds
    Between Step_3 and Confirm: {hours_spent_4 + minutes_spent_4 + seconds_spent_4: .2f} seconds""")
    
    return 

In [6]:
def tukeys_test_outliers(data):
    Q1 = data.quantile(0.25)
    Q3 = data.quantile(0.75)
    IQR = Q3 - Q1
    

    lower_q = Q1 - 1.5 * IQR
    upper_q = Q3 + 1.5 * IQR
    

    outliers = data[(data < lower_q) | (data > upper_q)] 
    
    return outliers

In [27]:
# Improved function for time spent in each step (with the function dt.total_seconds())

def time_steps(df):

    df["date_time"] = pd.to_datetime(df["date_time"])
    df_sort_date = df.sort_values(by = ["client_id", "date_time"])
    
    start_step1 = df_sort_date[ df_sort_date["process_step"].isin(["start","step_1"])]
    step1_step2 = df_sort_date[ df_sort_date["process_step"].isin(["step_1","step_2"])]
    step2_step3 = df_sort_date[ df_sort_date["process_step"].isin(["step_2","step_3"])]
    step3_confirm = df_sort_date[ df_sort_date["process_step"].isin(["step_3","confirm"])]
    
    
    time_spent = []
    
    for step_df in [start_step1, step1_step2, step2_step3, step3_confirm]:
        
        consecutive_duplicates = step_df.duplicated(subset=["client_id", "process_step"], keep = "first")
        step_df = step_df[~consecutive_duplicates]

        time_diff = step_df.groupby("client_id")["date_time"].diff().dt.total_seconds()
        time_diff = time_diff.dropna()
        
        outliers = tukeys_test_outliers(time_diff)

        time_diff_no_outliers = time_diff.iloc[~time_diff.index.isin(outliers.index)]
        average_time = time_diff_no_outliers.mean()
        time_spent.append(average_time)
    
    
    print(f"""The average activity duration of clients is:
    Between Start and Step_1: {time_spent[0]: .2f} seconds
    Between Step_1 and Step2: {time_spent[1]: .2f} seconds
    Between Step_2 and Step3: {time_spent[2]: .2f} seconds
    Between Step_3 and Confirm: {time_spent[3]: .2f} seconds""")
    
    return 

In [28]:
time_steps(df_exp)

The average activity duration of clients is:
    Between Start and Step_1:  21.41 seconds
    Between Step_1 and Step2:  35.42 seconds
    Between Step_2 and Step3:  85.15 seconds
    Between Step_3 and Confirm:  84.57 seconds


**Error rates**

In [ ]:
steps = df_exp.sort_values(by = ["client_id", "date_time"])
group = steps.groupby(["client_id", "process_step"], sort=False)

In [ ]:
steps["prev_step"] = steps.groupby("client_id")["process_step"].shift()

In [ ]:
steps.head()

In [ ]:
# First function to calculate error rates with loop 

def sequence_errors_loop(df):

    df_sort_date = df.sort_values(by = ["client_id", "date_time"])
    df_sort_date["prev_step"] = df_sort_date.groupby("client_id")["process_step"].shift()
    
    #df_sort_date["prev_step"] = df_sort_date["prev_step"].replace(["NaN", "nan"], np.nan)
    
    list_steps = ["start", "step_1", "step_2", "step_3", "confirm"]
    errors_count = 0
    total_steps = 0
    
    for client_id, row in df_sort_date.iterrows():
        current_step = row["process_step"]
        previous_step = row["prev_step"]
                    
        if pd.isna(previous_step):
            continue
        else:
        
            if previous_step is None and current_step != list_steps[0]:
                errors_count += 1
                
                
            elif previous_step is not None and previous_step != list_steps[4] and current_step != list_steps[list_steps.index(previous_step) + 1]:
                errors_count += 1
            
            total_steps += 1

            
    
        error_rate = (errors_count/total_steps)*100
    
        if errors_count <= 0:
            print(f"No errors in the activity of the client with id: '{client_id}'")
        else:
            print(f"The error rate for client with id '{client_id}' is: {error_rate: .2f}%")
        
    return         

In [30]:
# Improved function to calculate individual errors

def sequence_individual_errors(df):
    
    df_sort_date = df.sort_values(by = ["client_id", "date_time"])
    list_steps = ["start", "step_1", "step_2", "step_3", "confirm"]
    
    # previous step =  df_sort_date.groupby("client_id")["process_step"].shift()
    # current step = df_sort_date["process_step"]
    # same conditions of the previous "loop function"
    # +1 condition to specify the order that has to be followed (list_steps) in order not to have the error
    # always fillna() at the end because otherwise every time there is nan due to the end of the client_id group, 
    #  this will always "raise" the error and I want to avoid it 
    # list_steps.index(x) + 1) % len(list_steps => x is the previous step, +1 is the next step 
    #    (so we find the index of the next step in the list_steps); %len() allows to go back to the step with index 0 (start) 
    #    when the index overcome the max lenght of the list (so over the "confirm" step)
    
    errors = ((df_sort_date.groupby("client_id")["process_step"].shift().fillna("") == "") & (df_sort_date["process_step"] != list_steps[0])) | \
          (~df_sort_date.groupby("client_id")["process_step"].shift().fillna("").isin([None, list_steps[4], ""]) & \
           (df_sort_date["process_step"] != df_sort_date.groupby("client_id")["process_step"].shift(-1).fillna("")) & \
           (df_sort_date["process_step"] != df_sort_date.groupby("client_id")["process_step"].shift().map(lambda x: list_steps[(list_steps.index(x) + 1) % len(list_steps)] \
                                                                                                                            if not pd.isna(x) else "").fillna("")))
    
    df_errors = df_sort_date[errors] # errors is a boolean mask that allows to select only the rows of the sorted df where there are the errors specified in the mask 
   
    error_rate = ((df_errors.groupby("client_id").size() / df_sort_date.groupby("client_id").size())*100).fillna(0)
    
    for client, error in error_rate.items():
        
        if error <= 0:
            print(f"No errors in the activity of the client with id: '{client}'")
        else:
            print(f"The error rate for client with id '{client}' is: {error: .2f}%")

In [ ]:
df_reduced = df_exp.sort_values(by = ["client_id", "date_time"])[:500]

In [ ]:
df_sorted = df_exp.sort_values(by = ["client_id", "date_time"])

In [31]:
# Same function but to calculate overall error rate

def sequence_total_errors(df):
    
    df_sort_date = df.sort_values(by = ["client_id", "date_time"])
    list_steps = ["start", "step_1", "step_2", "step_3", "confirm"]
    
    errors = ((df_sort_date.groupby("client_id")["process_step"].shift().fillna("") == "") & (df_sort_date["process_step"] != list_steps[0])) | \
          (~df_sort_date.groupby("client_id")["process_step"].shift().fillna("").isin([None, list_steps[4], ""]) & \
           (df_sort_date["process_step"] != df_sort_date.groupby("client_id")["process_step"].shift(-1).fillna("")) & \
           (df_sort_date["process_step"] != df_sort_date.groupby("client_id")["process_step"].shift().map(lambda x: list_steps[(list_steps.index(x) + 1) % len(list_steps)] \
                                                                                                                            if not pd.isna(x) else "").fillna("")))
    df_errors = df_sort_date[errors] # errors is a boolean mask that allows to select only the rows of the sorted df where there are the errors specified in the mask 
   
    error_rate = ((df_errors.groupby("client_id").size() / df_sort_date.groupby("client_id").size())*100).fillna(0)
    
    # general error rate
    
    error_rate_total = (len(df_errors) / len(df_sort_date)) * 100

    if error_rate_total <= 0:
        print("No errors found in the entire dataset.")
    else:
        print(f"The overall error rate for the dataset is: {error_rate_total: .2f}%")

## **Redesign outcome**

### Completion rate for control and test group

In [38]:
df_final["Variation"] = df_final["Variation"].map(fun.replace_nan)

In [42]:
df_final_included = df_final[ df_final["Variation"].isin(["Test", "Control"])]

In [44]:
df_exp_final = pd.merge(df_final_included, df_exp, on = "client_id", how = "inner") 
# inner join to have only those clients that were in either control group or test group

In [45]:
df_exp_final.head()

,client_id,Variation,visitor_id,visit_id,process_step,date_time
0,9988021,Test,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:27:07
1,9988021,Test,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:26:51
2,9988021,Test,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:19:22
3,9988021,Test,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:19:13
4,9988021,Test,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:18:04


In [46]:
df_test = df_exp_final[df_exp_final["Variation"] == "Test"]
df_control = df_exp_final[df_exp_final["Variation"] == "Control"]

In [72]:
print(f"""The completion rate for the control group is: {completion_rate(df_control)}%
The completion rate for the test group is: {completion_rate(df_test)}%""")

The completion rate for the control group is: 65.6%
The completion rate for the test group is: 69.3%
